create framework for training seperate models for each location, and combining results before submission.  

In [120]:
import pandas as pd
import numpy as np
from functions import load_data, get_train_targets, get_test_data, prepare_submission

In [67]:
data_a, data_b, data_c = load_data()

X_train_a, targets_a = get_train_targets(data_a)
X_train_b, targets_b = get_train_targets(data_b)
X_train_c, targets_c = get_train_targets(data_c)

X_test_a, X_test_b, X_test_c = get_test_data()

To check that this work ok: run a random forest model on each.

In [92]:
drop_cols = ["date_calc", "time"]

In [99]:
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

class ColumnDropper(BaseEstimator, TransformerMixin):
    """Drops columns from the data."""

    def __init__(self, drop_cols = []):
        self.drop_cols = drop_cols

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_copy = X.copy()
        return X_copy.drop(columns=self.drop_cols)

data_process_pipeline = Pipeline([
    ('drop_cols', ColumnDropper(drop_cols=drop_cols)),
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)),
])

random_forest_pipeline = Pipeline([
    ('data_process', data_process_pipeline),
    ('random_forest', RandomForestRegressor(random_state=42))
])

In [102]:
random_forest_pipeline.fit(X_train_a, targets_a)
pred_a = random_forest_pipeline.predict(X_test_a.drop(columns=["id", "prediction", "location"]))

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- id
- location
- prediction


In [110]:
random_forest_pipeline.fit(X_train_b, targets_b)
pred_b = random_forest_pipeline.predict(X_test_b.drop(columns=["id", "prediction", "location"]))

In [111]:
random_forest_pipeline.fit(X_train_c, targets_c)
pred_c = random_forest_pipeline.predict(X_test_c.drop(columns=["id", "prediction", "location"]))

In [116]:
submission = prepare_submission(X_test_a, X_test_b, X_test_c, pred_a, pred_b, pred_c)

In [118]:
submission.to_csv('submissions/sepperate_models_rf.csv', index=False)

- this gave a score of 168